In [1]:
%load_ext autoreload
%autoreload 2

import os
import cv2
import matplotlib.pyplot as plt
from attention.img_proc.img_process import *
from attention.img_proc.img_split import *
from attention.models.face_models import *
from attention.utils.img_plot import *
from attention.utils.utilities import *
import copy

objc[37433]: Class CaptureDelegate is implemented in both /Users/noah/.pyenv/versions/3.10.6/envs/attention-env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1655824e8) and /Users/noah/.pyenv/versions/3.10.6/envs/attention-env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x13ba44860). One of the two will be used. Which one is undefined.
objc[37433]: Class CVWindow is implemented in both /Users/noah/.pyenv/versions/3.10.6/envs/attention-env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x165582538) and /Users/noah/.pyenv/versions/3.10.6/envs/attention-env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x119574a68). One of the two will be used. Which one is undefined.
objc[37433]: Class CVView is implemented in both /Users/noah/.pyenv/versions/3.10.6/envs/attention-env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x165582560) and /Users/noah/.pyenv/versions/3.10.6/envs/attention-env/lib/python3.10/site-packages/mediapip

In [125]:
image_path = '/Users/noah/Downloads/IMG_2069.jpg'

In [126]:
face = read_image(image_path)

In [117]:
mp_landmarks = find_landmarks(face)

In [118]:
landmark_list = convert_landmarks(face, mp_landmarks)
landmark_list

[(38, 27),
 (36, 24),
 (37, 25),
 (34, 21),
 (36, 23),
 (35, 22),
 (34, 18),
 (28, 20),
 (33, 16),
 (33, 15),
 (31, 10),
 (38, 27),
 (38, 27),
 (38, 27),
 (38, 28),
 (38, 28),
 (38, 28),
 (39, 29),
 (39, 30),
 (36, 24),
 (36, 25),
 (24, 17),
 (31, 20),
 (30, 20),
 (29, 21),
 (28, 21),
 (32, 20),
 (29, 18),
 (30, 18),
 (28, 19),
 (27, 19),
 (27, 21),
 (37, 33),
 (28, 20),
 (25, 22),
 (26, 21),
 (32, 24),
 (36, 27),
 (37, 28),
 (35, 28),
 (35, 28),
 (36, 28),
 (36, 28),
 (35, 30),
 (35, 24),
 (35, 23),
 (26, 19),
 (33, 21),
 (34, 24),
 (33, 24),
 (29, 25),
 (34, 22),
 (28, 17),
 (26, 18),
 (24, 15),
 (32, 17),
 (31, 18),
 (34, 29),
 (30, 31),
 (34, 25),
 (35, 25),
 (34, 29),
 (35, 29),
 (26, 18),
 (34, 25),
 (29, 17),
 (29, 16),
 (26, 12),
 (25, 16),
 (27, 14),
 (25, 19),
 (24, 18),
 (37, 27),
 (36, 28),
 (35, 28),
 (35, 25),
 (35, 29),
 (35, 29),
 (35, 29),
 (35, 24),
 (36, 28),
 (36, 28),
 (37, 28),
 (38, 30),
 (38, 29),
 (37, 29),
 (37, 28),
 (37, 28),
 (36, 28),
 (36, 29),
 (36, 29),

In [119]:
# GLOBAL PARAMS: LANDMARK INDICES
LEFT_EYE_EDGES =[362, 263]
RIGHT_EYE_EDGES =[33, 133]

LEFT_IRIS_CONTOUR = [474, 475, 476, 477]
RIGHT_IRIS_CONTOUR = [469, 470, 471, 472]

LEFT_IRIS_CENTER = [473]
RIGHT_IRIS_CENTER = [468]

NOSE = [1]

In [120]:
# Indices of the landmarks we focus on
landmark_idx = LEFT_EYE_EDGES  + LEFT_IRIS_CENTER + RIGHT_EYE_EDGES + RIGHT_IRIS_CENTER + NOSE

In [121]:
# Extracting the coordinates of the landmarks we focus on
landmark_focus_x = [landmark_list[idx] for idx in landmark_idx]
landmark_focus_x

[(37, 16), (41, 14), (39, 15), (28, 20), (32, 19), (30, 19), (36, 24)]

In [129]:
def score_head_movement(landmark_focus_x):
    nose_x, nose_y = landmark_focus_x[6]
    left_outside_corner_x, left_outside_corner_y = landmark_focus_x[1]
    right_outside_corner_x, right_outside_corner_y = landmark_focus_x[3]

    left_distance_eye_to_nose = np.sqrt((nose_x - left_outside_corner_x)**2 + (nose_y - left_outside_corner_y)**2)
    right_distance_eye_to_nose = np.sqrt((nose_x - right_outside_corner_x)**2 + (nose_y - right_outside_corner_y)**2)

    return left_distance_eye_to_nose / right_distance_eye_to_nose

score_head_movement(landmark_focus_x)

1.25

In [162]:
def direction_head_facing(landmark_focus_x):
    nose_x, nose_y = landmark_focus_x[6]
    left_outside_corner_x, left_outside_corner_y = landmark_focus_x[1]
    right_outside_corner_x, right_outside_corner_y = landmark_focus_x[3]

    left_distance_eye_to_nose = np.sqrt((nose_x - left_outside_corner_x)**2 + (nose_y - left_outside_corner_y)**2)
    right_distance_eye_to_nose = np.sqrt((nose_x - right_outside_corner_x)**2 + (nose_y - right_outside_corner_y)**2)
    
    if left_distance_eye_to_nose / right_distance_eye_to_nose < 0.65:
        return 'head left'
    elif left_distance_eye_to_nose / right_distance_eye_to_nose > 1.35:
        return 'head right'
    else:
        return 'head straight'

direction_head_facing(landmark_focus_x)


'head straight'

### IRIS

In [156]:
def left_detect_eye_direction(landmark_focus_x):
    # Calculate vector from left corner of left eye to right corner of left eye
    left_outside_x = landmark_focus_x[1][0]
    left_inside_corner_x= landmark_focus_x[0][0]
    left_eye_vector_x = left_outside_x - left_inside_corner_x 

    # Calculate position of left iris relative to left eye vector
    left_iris_x= landmark_focus_x[2][0]
    left_iris_vector_x = left_iris_x - left_inside_corner_x

    left_iris_position_relative_to_entire_eye = left_iris_vector_x / left_eye_vector_x

    if left_iris_position_relative_to_entire_eye < 0.63:
      return 'Looking straight'
    else:
      return 'Looking left'

left_detect_eye_direction(landmark_focus_x)


'Looking straight'

In [157]:
def right_detect_eye_direction(landmark_focus_x):
    # Calculate vector from left corner of left eye to right corner of left eye
    right_inside_x = landmark_focus_x[4][0]
    right_outside_corner_x = landmark_focus_x[3][0]
    right_eye_vector_x = right_inside_x - right_outside_corner_x

    # Calculate position of right iris relative to right eye vector
    right_iris_x = landmark_focus_x[5][0]
    right_iris_vector_x = right_inside_x - right_iris_x
    
    right_iris_position_relative_to_entire_eye = right_iris_vector_x / right_eye_vector_x

    if right_iris_position_relative_to_entire_eye < 0.6:
      return 'Looking straight'
    else:
      return 'Looking right'
    
right_detect_eye_direction(landmark_focus_x)

'Looking straight'

### ATTENTIVENESS NEW

In [158]:
def detect_attention(landmark_focus_x):
    left_eye = left_detect_eye_direction(landmark_focus_x)
    right_eye = right_detect_eye_direction(landmark_focus_x)
    head_direction = direction_head_facing(landmark_focus_x)

    if left_eye == right_eye and head_direction=='head straight':
        return 1  # attentive
    elif left_eye=='Looking left' and head_direction=='head right':
        return 1
    elif right_eye=='Looking right' and head_direction=='head left':
        return 1
    else:
        return 0  # inattentive
detect_attention(landmark_focus_x)

1

### FOR LOOP

In [159]:
def process_image(image_paths):
    results = []
    
    for image_path in image_paths:
        attention_score = -1
        face = read_image(image_path)
        mp_landmarks = find_landmarks(face)

        if mp_landmarks:
            landmark_list = convert_landmarks(face, mp_landmarks)

            LEFT_EYE_EDGES = [362, 263]
            RIGHT_EYE_EDGES = [33, 133]

            LEFT_IRIS_CENTER = [473]
            RIGHT_IRIS_CENTER = [468]
            
            NOSE = [1]

            landmark_idx = LEFT_EYE_EDGES + LEFT_IRIS_CENTER + RIGHT_EYE_EDGES + RIGHT_IRIS_CENTER + NOSE

            landmark_focus_x = [landmark_list[idx] for idx in landmark_idx]
            
            attention_score = detect_attention(landmark_focus_x)

            left_eye = left_detect_eye_direction(landmark_focus_x)
            right_eye = right_detect_eye_direction(landmark_focus_x)
        
            head_direction = direction_head_facing(landmark_focus_x)
            score_head_direction = score_head_movement(landmark_focus_x)

            #if left_eye is not None and right_eye is not None and left_eye == right_eye:
                #results.append((image_path, str(left_eye), str(left_iris_position_relative_to_entire_eye), str(right_eye), str(right_iris_position_relative_to_entire_eye), attention_score))
            #else:
                #results.append((image_path, str(left_eye), str(left_iris_position_relative_to_entire_eye), str(right_eye), str(right_iris_position_relative_to_entire_eye), attention_score))

            results.append((image_path, left_eye, right_eye, head_direction, score_head_direction, attention_score))
        

    return results

In [160]:
import os

directory_path = "/Users/noah/Desktop/images_lewagon/"

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    
    # Check if the file is an image file (you can add more extensions as needed)
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        
        # Construct the full file path
        image_path = os.path.join(directory_path, filename)
        
        # Process the image and get the results
        results = process_image([image_path])
        
        print(results)


[('/Users/noah/Desktop/images_lewagon/frame_33_4.png', 'Looking straight', 'Looking right', 'head left', 0.5676269839668552, 1)]
[('/Users/noah/Desktop/images_lewagon/frame_8_1.png', 'Looking straight', 'Looking right', 'head right', 1.3541957590563314, 0)]
[('/Users/noah/Desktop/images_lewagon/frame_11_3.png', 'Looking straight', 'Looking right', 'head straight', 1.1544196918377319, 0)]
[('/Users/noah/Desktop/images_lewagon/frame_37_0.png', 'Looking straight', 'Looking straight', 'head straight', 1.012220412723645, 1)]
[('/Users/noah/Desktop/images_lewagon/IMG_20230309_103423_4.jpg', 'Looking straight', 'Looking straight', 'head left', 0.6117076458450407, 0)]
[('/Users/noah/Desktop/images_lewagon/frame_13_1.png', 'Looking straight', 'Looking straight', 'head straight', 1.159423259677831, 1)]
[('/Users/noah/Desktop/images_lewagon/frame_35_2.png', 'Looking straight', 'Looking right', 'head straight', 0.7178447505439559, 0)]
[('/Users/noah/Desktop/images_lewagon/frame_13_0.png', 'Looking